In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/nto-places-description/vladimir_places_with_description.csv
/kaggle/input/nto-places-description/ekb_text_latents_name.pkl
/kaggle/input/nto-places-description/nn_places_with_description.csv
/kaggle/input/nto-places-description/yaroslavl_text_latents_name.pkl
/kaggle/input/nto-places-description/nn_image_latents.pkl
/kaggle/input/nto-places-description/ekb_image_latents.pkl
/kaggle/input/nto-places-description/ekb_places_with_description.csv
/kaggle/input/nto-places-description/vladimir_text_latents_name.pkl
/kaggle/input/nto-places-description/yaroslavl_image_latents.pkl
/kaggle/input/nto-places-description/yaroslavl_places_with_description.csv
/kaggle/input/nto-places-description/vladimir_image_latents.pkl
/kaggle/input/nto-places-description/nn_text_latents_name.pkl
/kaggle/input/nto-full/yaroslavl_images.csv
/kaggle/input/nto-full/EKB_places.csv
/kaggle/input/nto-full/Vladimir_images.csv
/kaggle/input/nto-full/Yaroslavl_places.csv
/kaggle/input/nto-full/EKB_images.csv

In [2]:
ekb_images = pd.read_csv('/kaggle/input/nto-full/EKB_images.csv')
ekb_places = pd.read_csv('/kaggle/input/nto-full/EKB_places.csv')

nn_images = pd.read_csv('/kaggle/input/nto-full/NN_images.csv')
nn_places = pd.read_csv('/kaggle/input/nto-full/NN_places.csv')

vladimir_images = pd.read_csv('/kaggle/input/nto-full/Vladimir_images.csv')
vladimir_places = pd.read_csv('/kaggle/input/nto-full/Vladimir_places.csv')

yaroslavl_images = pd.read_csv('/kaggle/input/nto-full/yaroslavl_images.csv')
yaroslavl_places = pd.read_csv('/kaggle/input/nto-full/Yaroslavl_places.csv')

yaroslavl_images = yaroslavl_images.rename({'img': 'image'}, axis=1)

def make_global_indexes(prefix, df_places):
    df_places = df_places.drop_duplicates(subset=['Name'])
    df_places = df_places.reset_index(drop=True)
    df_places['global_id'] = prefix + '_' + df_places.index.astype(str)
    return df_places

def make_local_index(prefix, df_images):
    df_images = df_images.drop_duplicates(subset=['image'])
    df_images = df_images.reset_index(drop=True)
    df_images['local_id'] = prefix + '_' + df_images.index.astype(str)    
    return df_images

# ekb
ekb_places = make_global_indexes('ekb', ekb_places)
ekb_images = make_local_index('ekb_local', ekb_images)
full_ekb = pd.merge(ekb_places, ekb_images, left_on='Name', right_on='name')[['Name', 'Kind', 'City', 'Lon', 'Lat', 'global_id', 'image', 'local_id']]
# nn
nn_places = make_global_indexes('nn', nn_places)
nn_images = make_local_index('nn_local', nn_images)
full_nn = pd.merge(nn_places, nn_images, left_on='Name', right_on='name')[['Name', 'Kind', 'City', 'Lon', 'Lat', 'global_id', 'image', 'local_id']]
# vladimir
vladimir_places = make_global_indexes('vlad', vladimir_places)
vladimir_images = make_local_index('v_local', vladimir_images)
full_vladimir = pd.merge(vladimir_places, vladimir_images, left_on='Name', right_on='name')[['Name', 'Kind', 'City', 'Lon', 'Lat', 'global_id', 'image', 'local_id']]
# yaroslavl
yaroslavl_places = make_global_indexes('y', yaroslavl_places)
yaroslavl_images = make_local_index('y_local', yaroslavl_images)
full_yaroslavl = pd.merge(yaroslavl_places, yaroslavl_images, left_on='Name', right_on='name')[['Name', 'Kind', 'City', 'Lon', 'Lat', 'global_id', 'image', 'local_id']]

# full_df = pd.concat([full_ekb, full_nn, full_vladimir, full_yaroslavl])
# full_df = full_df.reset_index(drop=True)

In [3]:
ekb = full_yaroslavl.copy()

### TEXT IMAGE SIMILARITY

In [4]:
!pip install ruclip==0.0.2 -q
!pip install -U huggingface_hub -q

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 2.18.0 requires huggingface-hub>=0.19.4, but you have huggingface-hub 0.2.1 which is incompatible.
tokenizers 0.15.2 requires huggingface_hub<1.0,>=0.16.4, but you have huggingface-hub 0.2.1 which is incompatible.
transformers 4.39.3 requires huggingface-hub<1.0,>=0.19.3, but you have huggingface-hub 0.2.1 which is incompatible.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ruclip 0.0.2 requires huggingface-hub==0.2.1, but you have huggingface-hub 0.22.2 which is incompatible.


In [5]:
import base64
from PIL import Image
from io import BytesIO

import torch
import ruclip

import torch.nn.functional as F

import joblib

In [6]:
model, processor = ruclip.load('ruclip-vit-base-patch32-384', device="cpu")

/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:669: FutureWarning: 'cached_download' is the legacy way to download files from the HF hub, please consider upgrading to 'hf_hub_download'
  warnings.warn(


bpe.model:   0%|          | 0.00/748k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/346 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/606M [00:00<?, ?B/s]

In [7]:
templates = ['{}', 'это {}', 'на фото {}']
predictor = ruclip.Predictor(model, processor, 'cpu', bs=8, templates=templates)

In [8]:
text_latents_name = joblib.load('/kaggle/input/nto-places-description/yaroslavl_text_latents_name.pkl')
image_latents = joblib.load('/kaggle/input/nto-places-description/yaroslavl_image_latents.pkl')

### get answers based on request

In [9]:
def get_request_embedding(request):
    request = [request]
    with torch.no_grad():
        text_latents = predictor.get_text_latents(request)
    
    return F.normalize(text_latents, p=2, dim=-1)

In [10]:
def get_cosine_similarity(request_latents, other_latents):
    cosine_similarities = torch.matmul(request_latents, other_latents.T)

    cosine_similarities = cosine_similarities.flatten()

    return cosine_similarities

In [11]:
def normalize_scores(scores):
    """Normalize scores to a [0, 1] range."""
    min_score = scores.min()
    max_score = scores.max()
    return (scores - min_score) / (max_score - min_score)

In [12]:
def filter_unique_places(sorted_indices, df):
    mapped_name_ids = df.iloc[sorted_indices.cpu().numpy()]['global_id']
    
    _, unique_indices = np.unique(mapped_name_ids, return_index=True)
    unique_indices_sorted = np.sort(unique_indices)  # Sort the indices to maintain the order
    
    final_indices = sorted_indices[unique_indices_sorted]
    
    return final_indices

In [15]:
def get_top_n_objects(df, indices_sorted, cosine_sim, top_n=5):
    top_indices = indices_sorted[:top_n].cpu().numpy()
    top_scores = cosine_sim[top_indices].cpu().numpy()
    
    top_data = [(df.loc[df.index == idx, 'local_id'].values[0], df.loc[df.index == idx, 'Name'].values[0], df.loc[df.index == idx, 'global_id'].values[0], score)
                for idx, score in zip(top_indices, top_scores)]
    
    return top_data

In [16]:
def get_top_n_on_request(request, df, text_latents_name, image_latents, top_n):
    text_latents_name = text_latents_name.cpu()
    image_latents = image_latents.cpu()
    
    text_latents = get_request_embedding(request)
    text_latents = text_latents.cpu()
    
    # cosine sim
    text_cosine_sim = get_cosine_similarity(text_latents, text_latents_name)
    image_cosine_sim = get_cosine_similarity(text_latents, image_latents)

    # normalization
    text_cosine_sim_norm = normalize_scores(text_cosine_sim)
    image_cosine_sim_norm = normalize_scores(image_cosine_sim)

    # uniting scores
    united_cosine_sim_norm = text_cosine_sim_norm + image_cosine_sim_norm

    united_cosine_sim_normalized = united_cosine_sim_norm / 2

    # sorting indices
    # text_sorted_indices = torch.argsort(text_cosine_sim_norm, descending=True)
    # image_sorted_indices = torch.argsort(image_cosine_sim_norm, descending=True)
    united_sorted_indices = torch.argsort(united_cosine_sim_normalized, descending=True)

    # filtering unique places
    final_sorted_indices = filter_unique_places(united_sorted_indices, df)
    
    top_data = get_top_n_objects(df, final_sorted_indices, united_cosine_sim_normalized, top_n)
    
    return top_data

In [17]:
from IPython.display import display

request = 'музей'
top_data = get_top_n_on_request(request, ekb, text_latents_name, image_latents, top_n=5)

for data in top_data:
    display(data[0])
    print(data[1:])


'y_local_1569'

('Музей древнерусского искусства', 'y_25', 0.93594337)


'y_local_1614'

('Музей зарубежного искусства', 'y_61', 0.9170064)


'y_local_1518'

('Музей Боевой Славы', 'y_79', 0.83543384)


'y_local_1645'

('Музей истории города Ярославля', 'y_19', 0.75056887)


'y_local_952'

('Дом муз', 'y_53', 0.7447806)
